In [3]:
import sys
import os
from pathlib import Path
import json

# Obtener la ruta absoluta a la carpeta raíz del proyecto
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))


# Añadir esa ruta a sys.path
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(ROOT)

# print(os.path.isfile(PROMPTS_DB_PATH))


c:\Formacion\Customer_bot


In [4]:
from bot.config import PROMPTS_DB_PATH, OPENAI_MODEL_NAME, OPENAI_API_KEY
from bot.models import BotState
from bot.nlp_agent import nlp_node
from bot.graph import initial_state

PROMPTS_DB_PATH = os.path.join(ROOT, PROMPTS_DB_PATH)

# Añadir esa ruta a sys.path
if PROMPTS_DB_PATH not in sys.path:
    sys.path.append(PROMPTS_DB_PATH)

print(PROMPTS_DB_PATH)

c:\Formacion\Customer_bot\db/prompts.db


In [5]:
path = Path(PROMPTS_DB_PATH)
if path.exists():
    with path.open("r", encoding="utf-8") as f:
        prompts = json.load(f)
        print("Claves en prompts:", list(prompts.keys()))
else:
    print("Fichero no encontrado.")

system_prompt = prompts["nlp_agent.system"]
prompt_answer = prompts["answer_agent.system"]

# print(prompt_nlp)
# print(prompt_answer)     

Claves en prompts: ['nlp_agent.system', 'answer_agent.system']


In [6]:
msg = "¿Dónde está mi pedido?"

initial_state_bot =  BotState(
    user_message=msg,
    nlp = None,
    knowledge_hits=[],
    answer=None,
    debug={}
)

print(initial_state_bot.model_dump_json(indent=4, ensure_ascii=False))

{
    "user_message": "¿Dónde está mi pedido?",
    "nlp": null,
    "knowledge_hits": [],
    "answer": null,
    "debug": {}
}


In [7]:
msg = "¿Dónde está mi pedido?"

bot_state = initial_state(msg)

print(type(bot_state))
print(bot_state.model_dump_json(indent=4, ensure_ascii=False))

<class 'bot.models.BotState'>
{
    "user_message": "¿Dónde está mi pedido?",
    "nlp": null,
    "knowledge_hits": [],
    "answer": null,
    "debug": {}
}


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from bot.models import NLPResult

In [22]:
# Creamos el LLM de LangChain (ChatOpenAI)

llm = ChatOpenAI(
    model=OPENAI_MODEL_NAME,
    temperature=0,
    api_key=OPENAI_API_KEY,
)
print(f"Conexión con OpenAI inicializada correctamente. Modelo: {OPENAI_MODEL_NAME}")
response = llm.invoke(
        [
            SystemMessage(content=system_prompt),
            HumanMessage(content=bot_state.user_message),
        ]
    )
raw = response.content


Conexión con OpenAI inicializada correctamente. Modelo: gpt-4.1-mini


In [26]:
data = json.loads(raw)

new_state = bot_state.model_copy()

new_state.nlp = NLPResult(**data)

debug = new_state.debug.copy()   # ← copia el dict para no tocar referencias
debug["nlp_raw"] = raw
new_state.debug = debug


In [27]:
print(type(new_state))
print(new_state.model_dump_json(indent=4, ensure_ascii=False))

<class 'bot.models.BotState'>
{
    "user_message": "¿Dónde está mi pedido?",
    "nlp": {
        "intent": {
            "tipo_mensaje": "pregunta",
            "intencion": "order_status",
            "confianza": 0.95,
            "sentimiento": "neutral"
        },
        "entidades": []
    },
    "knowledge_hits": [],
    "answer": null,
    "debug": {
        "nlp_raw": "{\n  \"intent\": {\n    \"tipo_mensaje\": \"pregunta\",\n    \"intencion\": \"order_status\",\n    \"confianza\": 0.95,\n    \"sentimiento\": \"neutral\"\n  },\n  \"entidades\": []\n}"
    }
}
